In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt')); print(fn_txt[:5])

In [ ]:
step = 5
for i in range(0, 802, step):
    print(f'reading [{i}, {i+step})', i)
    formatter = DataFormatter(fn_txt[i:i+step])
    print('cleaning up')
    formatter.cleanup()
    print('cleanup done')
    formatter.cycles1()
    for j, cyc in enumerate(formatter.cycles): cyc.to_feather(f'tmp/cycles/cycle{i+j}')

In [ ]:
%%time
formatter = DataFormatter(fn_txt[:80])
df_raw = formatter.df_raw

In [ ]:
%%time
formatter.cleanup()

In [ ]:
%%time
formatter.cleanup()

In [ ]:
formatter.cycles1()

In [ ]:
cols = [cyc[['推进速度', '刀盘扭矩', '刀盘功率', '总推进力', '主驱动1#电机输出功率', '刀盘转速']] for cyc in formatter.cycles]

In [ ]:
cycles = formatter.cycles
for i in range_of(cycles): cycles[i].to_feather(f'tmp/cycles/cycle{i}')

In [ ]:
cycles = formatter.cycles
for i in range_of(cycles): cycles[i].to_feather(f'tmp/cycles/cycle{i}')

In [ ]:
cycle_fns = Path('tmp/cycles/').glob('cycle*')
import feather
cycles = [feather.read_dataframe(str(o)) for o in cycle_fns]

In [ ]:
cont_names = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速', '主液压油箱温度']
dep_var = '桩号'
bidx = [beginning_index(c) for c in formatter.cycles]
cont = [o[cont_names].iloc[i:i+30] for i, o in zip(bidx, formatter.cycles)]

In [ ]:
cyc_cont = pd.DataFrame([o.values.flatten() for o in cont])
dep = pd.DataFrame({'桩号': [o['桩号'].iloc[0] for o in formatter.cycles]})
index = pd.Series(cyc_cont.index)
cyc_cont = pd.concat([index, cyc_cont, dep], axis=1)
cyc_cont.columns = list(range(cyc_cont.shape[1]-1)) + [dep_var]
print(cyc_cont.shape)
cyc_cont.head()

In [ ]:
procs = Normalize
databunch = TabularDataBunch.from_df('tmp', cyc_cont, dep_var, valid_idx=[3,4], bs=20, procs=procs); databunch

In [ ]:
x, y = next(iter(databunch.train_dl)); x

In [ ]:
learner = tabular_learner(databunch, [200, 200, 200])
learner.model

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit(50, 100)